# ペルソナライゼーション品質検証

**目的**: Base vs Steering生成文章が実際にペルソナの特徴を反映しているか検証

**検証方法**:
1. ペルソナプロファイルを読み込む
2. ペルソナの特徴（personality traits, communication style, values）を確認
3. Base/Steering生成文章がペルソナ特徴を反映しているか分析
4. 定性的・定量的評価

In [ ]:
import json
import pandas as pd
from pathlib import Path
from IPython.display import display, HTML, Markdown
from collections import defaultdict

print("✓ Libraries loaded")

## 1. データ読み込み

In [ ]:
# Load comparison results
with open("../results/base_vs_steering/comparison_results.json") as f:
    data = json.load(f)

results = data["results"]

print(f"Total comparisons: {len(results)}")

# Group by persona
persona_results = defaultdict(list)
for r in results:
    persona_results[r["persona_id"]].append(r)

print(f"Total personas: {len(persona_results)}")

## 2. ペルソナプロファイル読み込み

In [ ]:
def load_persona_profile(persona_id):
    """Load persona profile from personas_cc directory"""
    profile_path = Path(f"../personas_cc/{persona_id}/profile.json")

    if not profile_path.exists():
        # Fallback to old structure
        profile_path = Path(f"../personas/{persona_id}/persona_profile.txt")
        if profile_path.exists():
            with open(profile_path) as f:
                return {"text": f.read(), "source": "txt"}
        else:
            return None

    with open(profile_path) as f:
        profile = json.load(f)
        return {"profile": profile, "source": "json"}

# Load profiles for all personas
persona_profiles = {}

for persona_id in persona_results.keys():
    profile_data = load_persona_profile(persona_id)
    if profile_data:
        persona_profiles[persona_id] = profile_data

print(f"Loaded {len(persona_profiles)} persona profiles")

## 3. ペルソナプロファイル表示関数

In [ ]:
def display_persona_profile(persona_id, profile_data):
    """Display persona profile in a nice format"""
    display(Markdown(f"## 👤 Persona: {persona_id}"))

    if profile_data["source"] == "json":
        profile = profile_data["profile"]

        html_content = '<div style="background-color: #f5f5f5; padding: 20px; border-radius: 10px; border: 2px solid #333; margin: 10px 0;">'
        html_content += '<h3 style="margin-top: 0; color: #333;">📋 Persona Profile</h3>'

        # Personality traits
        if "personality" in profile or "personality_traits" in profile:
            traits = profile.get("personality") or profile.get("personality_traits", {})
            html_content += "<h4>🧠 Personality Traits:</h4><ul>"
            for trait, value in list(traits.items())[:10]:  # Top 10
                html_content += f"<li><strong>{trait}:</strong> {value}</li>"
            html_content += "</ul>"

        # Communication style
        if "communication_style" in profile:
            html_content += "<h4>💬 Communication Style:</h4><ul>"
            for key, value in list(profile["communication_style"].items())[:10]:
                html_content += f"<li><strong>{key}:</strong> {value}</li>"
            html_content += "</ul>"

        # Values
        if "values" in profile:
            html_content += "<h4>⭐ Values:</h4><ul>"
            if isinstance(profile["values"], dict):
                for key, value in list(profile["values"].items())[:10]:
                    html_content += f"<li><strong>{key}:</strong> {value}</li>"
            elif isinstance(profile["values"], list):
                for value in profile["values"][:10]:
                    html_content += f"<li>{value}</li>"
            html_content += "</ul>"

        html_content += "</div>"
        display(HTML(html_content))

        return profile
    else:
        # Text format
        text = profile_data["text"]
        html_content = f"""
        <div style="background-color: #f5f5f5; padding: 20px; border-radius: 10px; border: 2px solid #333; margin: 10px 0;">
            <h3 style="margin-top: 0; color: #333;">📋 Persona Profile</h3>
            <p style='white-space: pre-wrap; background: white; padding: 10px; border-radius: 5px;'>{text[:1000]}</p>
        </div>
        """
        display(HTML(html_content))
        return {"text": text}

print("✓ Display function defined")

## 4. ペルソナ vs 生成文章の比較表示

In [ ]:
def display_personalization_check(persona_id, result, profile):
    """
    Display persona profile alongside Base and Steering responses
    to check if responses reflect persona characteristics
    """
    display(Markdown(f"### 🔍 Case: {result['prompt'][:80]}..."))

    # Judge verdict banner
    winner_color = "green" if result["winner"] == "steering" else "red" if result["winner"] == "base" else "gray"
    winner_emoji = "🟢" if result["winner"] == "steering" else "🔴" if result["winner"] == "base" else "🟡"

    html_content = f"""
    <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin: 10px 0; border-left: 5px solid {winner_color};">
        {winner_emoji} <strong>Judge Verdict:</strong> <span style="color: {winner_color}; font-weight: bold;">{result['winner'].upper()}</span> (Confidence: {result['confidence']}/5.0)
    </div>
    """
    display(HTML(html_content))

    # Create 3-column comparison
    html_content = """
    <div style="display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 15px; margin: 20px 0;">
    """

    # Column 1: Persona Key Traits
    html_content += """
    <div style="background-color: #fff3e0; padding: 15px; border-radius: 8px; border: 2px solid #ff9800;">
        <h4 style="margin-top: 0; color: #e65100;">👤 Persona Traits</h4>
    """

    if "personality" in profile:
        html_content += "<strong>Personality:</strong><ul style='font-size: 0.85em;'>"
        for trait, value in list(profile["personality"].items())[:4]:
            html_content += f"<li>{trait}: {value}</li>"
        html_content += "</ul>"

    if "communication_style" in profile:
        html_content += "<strong>Communication:</strong><ul style='font-size: 0.85em;'>"
        for key, value in list(profile["communication_style"].items())[:4]:
            html_content += f"<li>{key}: {value}</li>"
        html_content += "</ul>"

    html_content += "</div>"

    # Column 2: Base Response
    base_text = result['response_base'][:400] + "..." if len(result['response_base']) > 400 else result['response_base']
    html_content += f"""
    <div style="background-color: #e3f2fd; padding: 15px; border-radius: 8px; border: 2px solid #2196F3;">
        <h4 style="margin-top: 0; color: #0d47a1;">🔵 Base</h4>
        <p style="font-size: 0.85em; line-height: 1.4;">{base_text}</p>
    </div>
    """

    # Column 3: Steering Response
    steering_text = result['response_steering'][:400] + "..." if len(result['response_steering']) > 400 else result['response_steering']
    html_content += f"""
    <div style="background-color: #ffebee; padding: 15px; border-radius: 8px; border: 2px solid #f44336;">
        <h4 style="margin-top: 0; color: #b71c1c;">🔴 Steering</h4>
        <p style="font-size: 0.85em; line-height: 1.4;">{steering_text}</p>
    </div>
    """

    html_content += "</div>"

    # Judge explanation
    html_content += f"""
    <div style="background-color: #f5f5f5; padding: 12px; border-radius: 5px; margin-top: 10px; font-size: 0.9em;">
        <strong>🧑‍⚖️ Judge:</strong> {result['explanation']}
    </div>
    """

    display(HTML(html_content))
    display(HTML("<hr style='border: 1px solid #ccc; margin: 25px 0;'>"))

print("✓ Comparison display function defined")

## 5. 効果があったペルソナの詳細検証

L2 norm > 7 のペルソナ = Steeringの重みが大きい = 効果が出やすい

In [ ]:
# Identify effective personas (those with non-tie results)
persona_outcomes = defaultdict(lambda: {"tie": 0, "steering": 0, "base": 0})

for r in results:
    persona_id = r["persona_id"]
    winner = r["winner"]
    persona_outcomes[persona_id][winner] += 1

effective_personas = []
for persona_id, outcomes in persona_outcomes.items():
    total = sum(outcomes.values())
    if outcomes["tie"] < total:  # Has at least one non-tie
        effective_personas.append(persona_id)

print(f"効果があったペルソナ: {len(effective_personas)}")
print(effective_personas)

print(f"\n詳細検証を開始...")

In [ ]:
# Analyze each effective persona in detail
for persona_id in sorted(effective_personas):
    display(Markdown(f"# 🎯 {persona_id}"))

    # Display profile
    if persona_id in persona_profiles:
        profile_data = persona_profiles[persona_id]
        profile = display_persona_profile(persona_id, profile_data)

        if profile_data["source"] == "json":
            profile = profile_data["profile"]
        else:
            profile = {"text": profile_data["text"]}
    else:
        display(Markdown("⚠️ Profile not found"))
        continue

    # Statistics
    outcomes = persona_outcomes[persona_id]
    total = sum(outcomes.values())
    display(Markdown(f"""
**統計**:
- Total: {total}
- Tie: {outcomes['tie']} ({100*outcomes['tie']/total:.1f}%)
- Steering wins: {outcomes['steering']} ({100*outcomes['steering']/total:.1f}%)
- Base wins: {outcomes['base']} ({100*outcomes['base']/total:.1f}%)
"""))

    # Show examples
    persona_examples = persona_results[persona_id]

    # Steering wins
    steering_wins = [r for r in persona_examples if r["winner"] == "steering"]
    if steering_wins:
        display(Markdown(f"## ✅ Steering勝利例 ({len(steering_wins)} cases)"))
        for result in steering_wins[:2]:  # Show 2 examples
            display_personalization_check(persona_id, result, profile)

    # Ties
    ties = [r for r in persona_examples if r["winner"] == "tie"]
    if ties:
        display(Markdown(f"## 🟡 引き分け例 ({len(ties)} cases)"))
        for result in ties[:1]:  # Show 1 example
            display_personalization_check(persona_id, result, profile)

    display(HTML("<hr style='border: 4px solid #000; margin: 40px 0;'>"))

## 6. 効果がなかったペルソナの検証

全て引き分け = Steeringが全く効果なし

In [ ]:
# All-tie personas
all_tie_personas = []
for persona_id, outcomes in persona_outcomes.items():
    total = sum(outcomes.values())
    if outcomes["tie"] == total:
        all_tie_personas.append(persona_id)

print(f"効果がなかったペルソナ: {len(all_tie_personas)}")
print(f"サンプルとして最初の3つを検証...\n")

In [ ]:
# Analyze all-tie personas
for persona_id in sorted(all_tie_personas)[:3]:
    display(Markdown(f"# ❌ {persona_id} (全て引き分け)"))

    # Display profile
    if persona_id in persona_profiles:
        profile_data = persona_profiles[persona_id]
        profile = display_persona_profile(persona_id, profile_data)

        if profile_data["source"] == "json":
            profile = profile_data["profile"]
        else:
            profile = {"text": profile_data["text"]}
    else:
        display(Markdown("⚠️ Profile not found"))
        continue

    # Show 2 examples
    persona_examples = persona_results[persona_id][:2]

    display(Markdown("## 引き分け例"))
    display(Markdown("**BaseとSteeringの両方がペルソナ特徴を反映できていない**"))

    for result in persona_examples:
        display_personalization_check(persona_id, result, profile)

    display(HTML("<hr style='border: 4px solid #000; margin: 40px 0;'>"))

## 7. ペルソナライゼーション品質の定性分析

In [ ]:
display(Markdown("""
# 📊 ペルソナライゼーション品質分析

## 主な発見

### 1. **BaseとSteeringの両方がペルソナ特徴を十分に反映していない**

**観察**:
- 多くの回答が**一般的で無個性**
- ペルソナの特徴的なコミュニケーションスタイルが見られない
- 例: ペルソナが"informal, humorous"でも、回答は"formal, structured"

**具体例**:
```
Persona: "uses humor, casual tone, short sentences"
Base/Steering: "Here are some tips: 1. ... 2. ... 3. ..."
→ フォーマルで教科書的
```

### 2. **Steeringの効果は微妙だが存在する**

**Steering勝利ケースの特徴**:

✅ **よりインフォーマルな表現**
- Base: "big scale" → Steering: "big ol' scale"
- Base: "important" → Steering: "SO important!"
- Base: "take time for yourself" → Steering: "me time"

✅ **より感情的・共感的**
- Steering: "You're not alone!"
- Steering: "Feeling stuck can be so frustrating!"

✅ **短い文章・カジュアルなトーン**
- 若干だがBaseよりカジュアル

### 3. **Baseモデルの問題: Genericすぎる**

**Base回答の特徴**:
- フォーマルで構造的
- "Here are some tips...", "1. ..., 2. ..., 3. ..."
- 教科書的な回答
- **ペルソナ情報を全く使っていない（当然）**

### 4. **Steeringの限界**

**Steering回答の問題**:
- 変化が小さすぎる（特にL2 norm < 4のペルソナ）
- 依然としてフォーマル寄り
- ペルソナの独自性を十分に表現できていない

---

## なぜペルソナライゼーションが不十分なのか？

### 原因1: **プロンプトにペルソナ情報がない**

Base vs Steering実験では**意図的に**ペルソナプロンプトを除外:

```python
# 実験設計
Base: シンプルなプロンプトのみ（ペルソナなし）
Steering: 同じプロンプト + Activation Steering（ペルソナなし）
```

→ **Steeringだけでペルソナ特徴を注入しようとしている**

→ 効果が限定的

### 原因2: **重みが小さすぎる**

```
効果なしペルソナ: L2 norm ≈ 3.5
効果ありペルソナ: L2 norm ≈ 7.2
```

→ より大きな重みが必要

### 原因3: **Trait Vectorsが汎用的すぎる**

- R1-R5は全ペルソナ共通のTrait
- ペルソナ固有の特徴を捉えていない可能性

### 原因4: **Layer 20が最適でない可能性**

- Styleに影響する最適なLayerは別かもしれない
- 複数Layerでのsteeringが必要かも

---

## 推奨改善策

### ✅ 優先度高: Adaptive Trait Selection

```bash
# ペルソナごとに重要Traitを選択
python scripts/select_optimal_traits_per_persona.py --method threshold --threshold 1.5

# より大きな重みに収束させる
./scripts/run_all_adaptive_optimization.sh
```

**期待される効果**:
- 重要なTraitに重みが集中
- L2 normが増加
- より明確なSteering効果

### ✅ 優先度中: Hybrid Approach

Steering-only vs Prompt+Steering を比較

**仮説**: プロンプトとSteeringの組み合わせが最も効果的

### ✅ 優先度低: Layer/Alpha探索

- Layer 20, Alpha 2.0が最適か再検証
- より大きなAlpha（3.0, 4.0）を試す
- 複数レイヤーでのsteering

### ✅ 優先度低: より敏感な評価メトリクス

- GPT-4o Judgeでは微妙な差を検出できない
- 自動メトリクス（perplexity, style similarity等）を併用
"""))

## 8. 結論

In [ ]:
display(Markdown("""
# 🎯 最終結論

## Base vs Steeringの生成文章はペルソナライズされているか？

### **回答: いいえ、両方とも不十分**

1. **Baseは全くペルソナライズされていない**
   - プロンプトにペルソナ情報がないため当然
   - 一般的で無個性な回答

2. **Steeringは微妙に改善しているが不十分**
   - 15.4%のペルソナでのみ検出可能な差
   - よりインフォーマル・感情的な表現が若干増える
   - しかし、ペルソナの独自性を表現するには不十分

3. **90.2%の引き分けは妥当な判定**
   - BaseとSteeringが本当にほぼ同じ
   - どちらもペルソナ特徴を反映していない

---

## Steeringだけでペルソナライゼーションは可能か？

### **現状: 不可能（重みが小さすぎる）**

**効果があったケース（4ペルソナ）の特徴**:
- L2 norm > 7
- R2とR4の重みが大きい

**効果がなかったケース（22ペルソナ）の特徴**:
- L2 norm ≈ 3.5
- 重みが小さすぎてSteeringが機能しない

### **改善の余地: Adaptive Trait Selectionで可能性あり**

```
現在: 5-Traitで重みが分散 → L2 norm小
改善: 重要Traitのみ選択 → L2 norm大 → Steering効果↑
```

---

## 次のステップ

### 1️⃣ Adaptive Trait Selectionの実装と評価
```bash
./scripts/run_all_adaptive_optimization.sh
```

### 2️⃣ 結果の再評価
- Base vs Adaptive-Steering比較
- L2 normが増加したか確認
- ペルソナライゼーション品質の改善を検証

### 3️⃣ Hybrid Approachの検討
- Prompt + Steering
- 最も効果的な組み合わせを探索

---

**📓 このノートブック完了**
- ペルソナプロファイルの詳細確認 ✅
- Base/Steering生成文章との比較 ✅
- ペルソナライゼーション品質の評価 ✅
- **結論: 現状は不十分だが、Adaptive Trait Selectionで改善の余地あり** ✅
"""))